In [2]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader,random_split,TensorDataset
from torchvision import datasets,transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [3]:
df=pd.read_csv(r"C:\Users\saipr\Downloads\creditcard.csv\creditcard.csv")
df.dropna(inplace=True)
df.drop('Time',axis=1,inplace=True)
x=df.drop(columns=['Class'])
y=df['Class']
#print(y.shape,x.shape)
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)
smote=SMOTE(random_state=2)
x_train_smote,y_train_smote=smote.fit_resample(x_train,y_train)
print(sum(y_train==1),sum(y_train==0),"\n",sum(y_train_smote==1),sum(y_train_smote==0))
x_train_smote=torch.tensor(np.array(x_train_smote),dtype=torch.float32)
y_train_smote=torch.tensor(np.array(y_train_smote),dtype=torch.int64)
x_train=torch.tensor(np.array(x_train),dtype=torch.float32)
x_test=torch.tensor(np.array(x_test),dtype=torch.float32)
y_train=torch.tensor(np.array(y_train),dtype=torch.int64)
y_test=torch.tensor(np.array(y_test),dtype=torch.int64)
train_data_smote=TensorDataset(x_train_smote,y_train_smote)
train_loader_smote=DataLoader(train_data_smote,batch_size=64,shuffle=True)
train_data=TensorDataset(x_train,y_train)
test_data=TensorDataset(x_test,y_test)
train_loader=DataLoader(train_data,batch_size=64,shuffle=True)
test_loader=DataLoader(test_data,batch_size=64,shuffle=True)


394 227451 
 227451 227451


In [4]:
class NN(nn.Module):
    def __init__(self,input_size=29,output_size=2):
        super().__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,128),
            nn.ReLU(),
            nn.Linear(128,256),
            nn.ReLU(),
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,16),
            nn.ReLU(),
            nn.Linear(16,4),
            nn.ReLU(),
            nn.Linear(4,2)
        )
    def forward(self,x):
        return self.network(x)

In [5]:
model=NN()
criterion=nn.CrossEntropyLoss()
optimiser=optim.Adam(model.parameters(),lr=1e-3)
model_smote=NN()
num_epochs=20

In [6]:
print('This output is on training data which is after smoting the data')
for epoch in range(num_epochs):
    for input,output in train_loader_smote:
        pred_out=model_smote(input)
        # print(pred_out.size())
        loss=criterion(pred_out,output)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f'Loss in epoch [{epoch+1}/{num_epochs}]:{loss}')
    if loss<10e-07:
         break

This output is on training data which is after smoting the data
Loss in epoch [1/20]:0.7091937065124512
Loss in epoch [2/20]:0.7155966758728027
Loss in epoch [3/20]:0.687050998210907
Loss in epoch [4/20]:0.7150242328643799
Loss in epoch [5/20]:0.7029466032981873
Loss in epoch [6/20]:0.6896953582763672
Loss in epoch [7/20]:0.6840416193008423
Loss in epoch [8/20]:0.6968116164207458
Loss in epoch [9/20]:0.702636182308197
Loss in epoch [10/20]:0.7092267274856567
Loss in epoch [11/20]:0.6900562047958374
Loss in epoch [12/20]:0.6967028379440308
Loss in epoch [13/20]:0.6897612810134888
Loss in epoch [14/20]:0.7215195894241333
Loss in epoch [15/20]:0.7029429078102112
Loss in epoch [16/20]:0.6871228218078613
Loss in epoch [17/20]:0.7093034386634827
Loss in epoch [18/20]:0.7000370621681213
Loss in epoch [19/20]:0.7183034420013428
Loss in epoch [20/20]:0.702218234539032


In [89]:
for epoch in range(num_epochs):
    for input,output in train_loader:
        pred_out=model(input)
        # print(pred_out.size())
        loss=criterion(pred_out,output)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f'Loss in epoch [{epoch+1}/{num_epochs}]:{loss}')
    if loss<10e-07:
         break

Loss in epoch [1/20]:0.022506870329380035
Loss in epoch [2/20]:0.0043348162434995174
Loss in epoch [3/20]:0.001961809117347002
Loss in epoch [4/20]:0.0016750365030020475
Loss in epoch [5/20]:0.0018130785319954157
Loss in epoch [6/20]:0.001731921685859561
Loss in epoch [7/20]:0.0017503669951111078
Loss in epoch [8/20]:0.0017673838883638382
Loss in epoch [9/20]:0.0016707520699128509
Loss in epoch [10/20]:0.0017010994488373399
Loss in epoch [11/20]:0.0016975293401628733
Loss in epoch [12/20]:0.0017573880031704903
Loss in epoch [13/20]:0.001696815132163465
Loss in epoch [14/20]:0.001733230659738183
Loss in epoch [15/20]:0.001768692978657782
Loss in epoch [16/20]:0.0017077637603506446
Loss in epoch [17/20]:0.0017268045339733362
Loss in epoch [18/20]:0.001774523756466806
Loss in epoch [19/20]:0.0017876134952530265
Loss in epoch [20/20]:0.0017303746426478028


In [9]:
def accuracy(model,dataloader,classes):
    model.eval()
    with torch.no_grad():
        y_test,y_pred=[],[]
        for input,output in dataloader:
            pred_output=model(input)
            y_test.extend(output.numpy())
            y_pred.extend(torch.argmax(pred_output,dim=1).numpy())
        #classes=dataloader.classes
        cor,tot=[],[]
        for i in range(len(classes)):
            cor.append(sum(1 for j,k in zip(y_test,y_pred) if j==i and k==i))
            tot.append(sum (1 for j in y_test if j==i))
        print(f'accuracy report of {dataloader}:')
        for i,cl in enumerate(classes):
            print(f'{cl}:[{cor[i]}/{tot[i]}]')
        print(classification_report(y_test,y_pred))
    model.train()
print(accuracy(model_smote,train_loader_smote,['Non Fraud','Fraud']))

accuracy report of <torch.utils.data.dataloader.DataLoader object at 0x00000208E09C1F70>:
Non Fraud:[227341/227451]
Fraud:[0/227451]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67    227451
           1       0.00      0.00      0.00    227451

    accuracy                           0.50    454902
   macro avg       0.25      0.50      0.33    454902
weighted avg       0.25      0.50      0.33    454902

None
